이번 장에서는 데코레이터를 사용해, 이미 존재하는 함수를 기반으로 새로운 함수를 만드는 방법을 살펴본다. 또한 functools 모듈에 있는 두 가지 함수를 좀 더 살펴본다. 그 둘은 update_wrapper()와 wrap()으로 데코레이터를 만드는 것을 도와준다.

데코레이터 함수의 이점 중 하나는 합성 함수를 만들 수 있다는 것이다. 이 함수는 여러 근원으로부터 기능을 가져올 수 있다. 합성 함수인 $f∘g(x)는 $f(g(x))$보다 복잡한 알고리즘을 표현할 때 좀 더 이해하기 쉽다. 복잡한 처리를 표현하는 경우 여러 가지 구문적인 대안이 존재하는 것이 도움이 되는 경우가 자주 있다.

### 고차 함수로서의 데코레이터

데코레이터의 핵심 아이디어는 기존의 함수를 다른 형태로 변환하는 것이다. 데코레이터는 데코레이터 자신과 데코레이션 대상이 되는 원래의 함수를 가지고 일종의 합성 함수를 만들어 낸다.

데코레이터 함수를 사용하는 방법은 다음 두 가지 중 하나다.

* 다음과 같이 함수 앞에 접두사로 사용하면 기반 함수와 같은 이름의 새로운 함수를 정의할 수 있다.

In [3]:
def decorator(func):
    pass
@decorator
def ori_func():
    pass

* 새로운 함수를 반환하는 연산으로 명시적으로 사용할 수 있다. 이 경우 만들어지는 함수에 새로운 이름을 부여할 수도 있다.

In [4]:
def ori_func():
    pass
original_func2 = decorator(ori_func)

이 둘은 같은 연산을 두 가지 다른 구문으로 사용한 것이다. 접두사 표기법의 장점은 간결하고 깔끔하다는 것이다. 일부 독자들은 접두사로 사용할 때 이를 더 잘 인식한다. 두 번째 방식은 명시적이며 더 유연하다. 접두사 방식을 더 많이 사용하지만 두 번째 방식을 사용해야만 하는 경우도 있다. 새로운 함수가 원래의 함수를 대치하는 것을 우너하지 않는 경우가 그러하다. 다음과 같은 명령을 사용하면 원래 함수와 데코레이션한 결과 함수를 함께 사용할 수 있을 것이다.

In [5]:
new_function = decorator(ori_func)

파이썬 함수들은 일급 계층 객체다. 함수를 인자로 받거나 함수를 결과로 반환하는 함수는 분명 파이썬 언어에 내장된 기능이다. 일때 열린 질문 하나는 어떻게 어떤 함수의 내부 코드 구조를 갱신하거나 조정할 수 있는가? 라는 것이다. 

그에 대한 답은 "그렇게 하지 않는다"이다.

코드의 내부를 더럽히는 대신, 원래의 함수를 둘러싸는 새 함수를 정의하는 것이 훨씬 깔끔하다. 데코레이터를 정의하는 경우, 다음과 같은 두 단계의 고차 함수가 관련된다.

* 데코레이터 함수는 래퍼함수를 기반 함수에 적용하여 새로운 래퍼 함수를 반환한다. 이 함수는 데코레이션된 함수를 만들어 내는 과정에서 단 한 번 실행되는 계산을 수행할 수 있다.

* 래퍼 함수는 기반 함수를 평가할 수 있다. 이 함수는 데코레이션된 함수가 평가될 때마다 평가될 것이다.

다음은 간단한 데코레이터 예제다.

In [7]:
from functools import wraps

def nullable(function):
    @wraps(function)
    def null_wrapper(arg):
        return None if arg is None else function(arg)
    return null_wrapper

거의 대부분의 경우 functools.wraps() 함수를 사용해 데코레이션도니 함수가 원래 함수의 애트리뷰트를 유지하도록 보장한다. 예를 들어 \__namn__과 \__doc__ 애트리뷰트를 복사하면 데코레이션된 함수가 원래 함수와 같은 문서화 문자열과 이름을 가지게 할 수 있다.

데코레이터는 메타 프로그래밍, 즉 코드를 만들어 내는 코드다. 하지만 실제 데이터를 처리할 때는 데코레이터가 반환한 래퍼 함수를 사용할 것이다. 

만든 @nullable 데코레이터를 다음과 같이 적용해 합성 함수를 만들 수 있다.

In [9]:
import math

nlog = nullable(math.log)

In [10]:
some_data = [10, 100, None, 50, 60]
scaled = map(nlog, some_data)

In [11]:
list(scaled)

[2.302585092994046,
 4.605170185988092,
 None,
 3.912023005428146,
 4.0943445622221]

함수를 데이터 값의 컬렉션에 적용했다. None인 원소도 문제 없이 None이라는 결과를 내놓는다. 이 과정에서 예외 처리 등을 사용하지 않았다.

*사실 이 예제는 단위 테스트에 적합하지 않다. 부동 소수점 수를 테스트하려면 값을 반올림 할 필요가 있을 것이다. 이를 위해쇼ㅓ는 널을 인식하는 round() 함수를 만들 필요가 있다.

다음은 데코레이터 표기법을 사용해 널을 인식하는 round() 함수를 만드는 방법을 보여준다.

In [12]:
@nullable
def nround(x):
    return round(x, 4)

이 함수는 round() 함수를 부분 적용한 후 널을 인식하도록 감싼 것이다. 어떤 측면에서 이 함수는 파이썬이 프로그래머들에게 이미 제공 중인 함수형 프로그래밍 기능 중에서 상대적으로 복잡한 부분이라 할 수 있다. 

다음과 같이 널을 인식하는 round() 함수를 만들 수도 있다.

In [13]:
nround4 = nullable(lambda x:round(x,4))

### functools의 update_wrapper() 함수 사용하기 

@wraps 데코레이터는 update_wrapper() 함수를 사용해 감싸진 함수의 애트리뷰트를 보존한다. 일반적으로, 우리의 기본적인 필요에는 그 데코레이터만으로 충분하다. 이 함수는 원래의 함수에서 특정 애트리뷰트 목록을 복사해 데코레이터가 만들어 내는 함수에 집어넣는다. 어떤 구체적인 애트리뷰트들을 복사할까? 그러한 정의는 모듈 전역 변수에 있다.

update_wrapper() 함수는 모듈 전역 변수를 사용해 어떤 애트리뷰트를 보존할 것인지를 결정한다. WRAPPER_ASSIGNMENRS 변수는 기본적으로 복사해야 할 애트리뷰트들을 정의한다. 그 변수의 디폴트 값은 다음과 같다.

('\__module__', '\__name__', '\__qualname__', '\__doc__', '\__annotation__')

이 목록을 쓸모 있게 변경하는 것은 쉽지 않다. 애트리뷰트를 추가로 복사하려면 우리가 사용할 함수들이 그러한 추가 애트리뷰트를 가지고 있다는 것을 확신할 수 있어야 한다. 하지만 이를 보장하는 것은 간단한 문제가 아니다. def문의 내부가 일반 프로그래머들이 쉽게 변경할 수있게 열려 있지 않기 때문이다.

새 애트리뷰트를 구겨 넣는 것이 쉽지 않기 때무에 함수를 감싸는 동작을 정당하게 확장하거나 변경해야 하는 것은 쉽지 않다. 따라서 보통은 WRAPPER_ASSIGNMENT 변수를 참조 목적으로만 사용하곤 한다.

하지만 Callable 객체를 사용하려면 호출 가능 클래스의 정의에 특정 애트리뷰트를 추가할 수 있다. 그러한 경우 데코레이터가 원래의 Callable 객체를 감싸면서 그러한 애트리뷰트를 복사해야 하는 상황이 생길 수도 있다. 하지만 그러한 경우에도 복잡한 데코레이터 기법을 사용하기보다는 클래스 저으이 자체를 변경하여 같은 목적을 달성하는 편이 좀 더 간단해 보인다.

### 횡단 관심사

데코레이터 뒤에 숨셔진 한 가지 일반적인 원칙은, 데코레이터를 적용할 대상 함수와 데코레이터로부터 합성 함수를 만들 수 있다는 것이다. 

여러 함수에 걸쳐 적용할 수 있기 때문에 이러한 관심사를 횡단 관심사라고 부르기도 한다. 이들은 데코레이터 형태로 설계하여 구현한 후 애플리케이션이나 프레임워크 전반에 걸쳐 관심의 대상이 되는 모든 클래스에 적용할 수 있는 요소들이다.

앞에서 설명한 것처럼 집중화하면 좋은 관심하세응 다음과 같은 것들이 있다.

* 로그

* 감사

* 보안

* 불완전한 데이터 처리

### 합성 설계

합성 함수에 대한 일반적인 수학 표기법은 다음과 같다.

$$f∘g(x) =  f(g(x))$$

파이썬에서 이와 같은 경우를 여러 줄로 표현하면 다음과 같다.

In [15]:
def f(func):
    pass
@f
def g(x):
    pass

위 코드는 $f∘g(x)$와 동등하지 않다. 동등성이 정확히 성립하지 않는 이유는 @f가 $f(y)와 g(x)를 합성하는 수학적인 추상화와 같지 않기 때문이다. 함수의 합성을 기술하려는 목적에서 살펴볼 때 앞으로는 f(y)에 대한 추상화와 @f 데코레이터의 구현상의 차이는 무시할 것이다.

데코레이터가 다른 함수를 감싸기 때문에 파이썬에서는 좀 더 일반적인 합성을 제공한다. 래퍼 함수는 보통 다음과 같은 모양을 띄고 있다.

In [18]:
@wraps(arg_func)
def something_wrapper(*args, **kw):
    result = arg_func(*args, **kw)

### 잘못된 데이터 처리하기

탐색적 자료 분석의 횡단 관심사 중 하나는 데이터가 없거나 구문 분석할 수 없는 수치 값을 어떻게 처리할 것인지다. 종종 float, int, Decimal 통화 값을 일관된 방법으로 처리할고 싶을 때가 있다. 

In [24]:
import decimal
def bad_data(function):
    @wraps(function)
    def wrap_bad_data(text, *args, **kw):
        try:
            return function(text, *args, **kw)
        except (ValueError, decimal.InvalidOperation):
            cleaned = text.replace(",", "")
            return function(cleaned, *args, **kw)
    return wrap_bad_data

이 함수는 주어진 변환 함수를 감싸 잘못된 값이 있는 경우 두 번째 변환을 시도하도록 만든다. None 값을 "적용 불가능"을 표현하는 코드로 사용하는 경우라면, 예외 처리 시 None을 반환하면 될 것이다. 

이 래퍼는 다음과 같이 사용할 수 있다.

In [21]:
bd_int = bad_data(int)

In [25]:
bd_float = bad_data(float)
db_decimal = bad_data(decimal.Decimal)

이렇게 하면 정상적인 데이터를 처리할 뿐만 아니라 한계는 있지만 잘못된 데이터 중 일부도 정리하여 정상적으로 처리할 수 있다. 

In [26]:
bd_int('13')

13

In [27]:
bd_int('1,371')

1371

In [29]:
bd_int('1,371', base = 16)

4977

### 매개변수를 데코레이터에 추가하기

In [30]:
def deco(arg):
    pass

@deco
def func():
    pass

살펴봤던 잘못된 데이터 처리 데코레이터를 확장하여 좀 더 유연하게 데이터 변환을 처리하도록 만들 수 있다. 제거할 문자들을 매개변수로 받는 데코레이터를 정의할 것이다. 다음은 그렇게 매개변수화한 데코레이터다.

In [32]:
import decimal
def bad_char_remove(*char_list):
    def cr_decorator(function):
        @wraps(function)
        def wrap_char_remove(text, *args, **kw):
            try:
                return function(text, *args, **kw)
            except (ValueError, decimal.InvalidOperation):
                cleaned = clean_list(text, char_list)
                return function(cleaned, *args, **kw)
        return wrap_char_remove
    return cr_decorator

이 데코레이터를 사용해 다음과 같은 변환 함수를 만들 수 있다.

In [33]:
@bad_char_remove("$", ",")
def currency(text, **kw):
    return decimal.Decimal(text, **kw)

In [34]:
currency("13")

Decimal('13')

In [36]:
currency("3.14")

Decimal('3.14')

### 설계의 한계를 인식하기

데이터를 정리하는 경우, 이상한 문자를 단순히 제거하는 것만으로는 충분치 않을 수도 있다. 지리적인 위치 정보를 다루는 경우를 예로 들면, 다양한 형식이 있을 수 있다.

간단한 고차 함수도 좋은 생각이다. 반면, 데코레이터는 이러한 경우 끔찍할 정도로 제대로 작동하지 ㅇ낳는다. 데코레이터가 의미가 있도록 해주는 몇 가지 제약 사항을 살펴보기 위해 데코레이터를 기반으로 하는 설계를 살펴본다.

설계 요구사항에는 다음과 같이 서로 직교적인(orthogonal) 두 가지 선택 사항이 있다.

1. 출력 변환(int, float, Decimal)
2. 입력 정리(이상한 문자 없애기, 좌표 형식 표준화하기)

이상적인 경우, 이 두 가지 관심사 중 하나는 감싸야 할 대상인 필수적인 함수여야 하고, 다른 하나는 래퍼를 통해 포함시켜야 하는 어떤 것이 되어야 할 것이다. 어떤 것이 필수적인 함수이고, 어떤 것이 래퍼에 들어가야 하는 것인지는 분명하지 않다. 그러한 구분이 불분명한 이유 중 한 가지는 앞에서 살펴본 에제가 단순히 두 가지 부분으로 이뤄진 함수 합성보다 좀 더 복잡하기 때문이다.

앞의 예제는 실제로 세 가지 부분의 합성을 만들어 낸다.

1. 출력 변환 

2. 입력 정리

3. 변환을 시도하는 삼수, 예외가 발생하면 입력을 정리한 후에 다시 변환을 시도 

다음과 같이 변환 함수에 대한 데코레이터의 인자로 정리 함수를 포함시킬 수 있을 것이다.

In [37]:
@cleanse_before(cleanser)
def convert(text):
    something

다음과 같이 변환 함수를 정리 함수에 대한 데코레이터의 인자로 포함시킬 수도 있을 것이다.

In [38]:
@then_convert(converter)
def clean(text):
    something

여기서는 대부분의 경우 내장 변환 함수를 활용하기 때문에 @then_convert 스타일의 데코레이터를 사용할 수 있을 것이다. 하지만 어떤 방식을 선택할 것인지는 명확하지 않다.

데코레이터는 다음과 같을 것이다.

In [39]:
def then_convert(convert_function):
    def clean_convert_decorator(clean_function):
        @wraps(clean_function)
        def cc_wrapper(text, *args, **kw):
            try:
                return convert_function(text, *args, **kw)
            except (ValueError, decimal.InvalidOperation):
                cleaned = clean_function(text)
                return convert_function(cleaned, *args, **kw)
        return cc_wrapper
    return clean_convert_decorator

세 가지 계층으로 이뤄진 데코레이터를 만들었다. 가장 중심에는 convert_function을 적용하는 cc_wrapper() 함수가 있다. 변환에 실패하는 경우에는 clean_function을 호출한 후 다시 convert_function을 호출한다. 이 함수는(cc_wrapper)는 clean_convert_decorator라는 구체적인 데코레이터를 통해 clean_function을 감싼다. 다시 그 구체적인 데코레이터에는 convert_function이라는 자유 변수가 있다. 구체적인 데코레이터는 데코레이터 인터페이스인 then_convert()에 변환 함수를 전달하여 만들어진다.

이제 좀 더 유연한 변환 및 정리 함수를 다음과 같이 만들 수 있다.

In [43]:
@then_convert(int)
def drop_punct(text):
    return text.replace(",", "").replace("$", "")

이 정수 변환은 주어진 정리 함수(drop_punct)에 데코레이터를 적용한 거싱다. 여기서 정리 함수는 , $ 를 제거한다. 정수 변환은 이렇게 정리한 결과를 감싼다. 


In [44]:
drop_punct("1,701")

1701

이러한 방식을 통해 복잡한 정리 함수와 변환을 매구 깔끔하게 한데 엮을 수 있지만, 결과는 혼란스러우 수 도 있다. 함수의 이름은 내부 핵심 정리 알고리즘의 이름이 된다. 전체 합성 함수에 기여하는 다은 함수들의 존재는 이름에 드러나지 않는다.

이에 대한 대안으로, 다음과 같이 정수 변환을 정의할 수 있다.

In [45]:
def drop_punct(text):
    return text.replace(",", "").replace('$', '')
drop_punct_int = then_convert(int)(drop_punct)

이렇게 하면 데코레이션한 정리 함수에 새로운 이름을 부여할 수 있다. 이름과 관련된 문제도 해결해준다. 하지만 함수를 만드는 부분에서 내부를 너무 노출한다고 할 수 있다. 

여기서 한계에 도달했다고 말할 수 있다. 데코레이터는 이러한 종류의 설계에 적합하지 않다. 일반적으로, 어떤 함수에 고정적이고, 단순한 관심사를 몇 가지 추가하려 하는 경우에는 데코레이터가 잘 작동한다. 또한 데코레이터는 추가하려는 관심사를 애플리케이션 코드의 의미에 있어 핵심적인 부분이 아니라 하부 구조나 지원 기능으로 생각할 수 있는 경우에도 중요하다.

여러 가지 직교적인 차원이 관련된 문제의 경우, 최종 결과물인 호출 가능한 객체는 여러 가지 종류의 전략 객체를 끼워넣을 수 있는 것이어야한다. 그래야 더 만족스러운 결과를 얻을 수있다. 고차 함수를 만드는 부분을 더 자세히 살펴봐야 할 것이다. 그 후 고차 함수를 위한 매개변수의 다양한 조합을 사용한 부분 적용 함수를 만들 수 있을 것이다. 

로그나 보안 검사를 수행하는 것은 문제와 직접적인 견롼이 없는 백그라운드 작업의 전형적인 예라 할 수 있다. 작성하느 ㄴ코드 전체에 편재해야 하는 처리가 있는 경우라면, 데코레이터가 좀 더 적합하다.